# Предсказание ухода клиента для банка "Бета-Банк"
## Постановка задачи

"Бета-Банк" решает проблему оттока клиентов. Согласно расчетов маркетологов банка, сохранять текущих клиентов дешевле, чем привлекать новых. 

Нужно научиться прогнозировать, уйдёт клиент из банка в ближайшее время или нет. Исходные данные для анализа - исторические данные о поведении клиентов и расторжении договоров с банком.

Нужно построить прогнозирующую модель с предельно большим значением F1-меры (не менее 0.59), затем проверить F1-меру на тестовой выборке. Дополнительно нужно измерять метрику AUC-ROC и сравнить её значения с F1-мерой.


## Импорт библиотек и данных
Импортируем датасет в переменную `df` и ознакомимся с данными. 

Исходные данные представляют собой датасет на 10_000 строк и 14 столбцов. Названия столбцов в CamelCase. Три столбца (`RowNumber`, `CustomerId`, `Surname`) неинформативны, их нужно будет удалить. Типы данных адекватны, но можно преобразовать колонки `HasCrCard` и `IsActiveMember` из численного типа в логический. В колонке `Tenure` есть пустые значения, от которых нужно будет избавиться.

В целом данные адекватны и пригодны для анализа после предобработки.

In [3]:
from pathlib import Path

import pandas as pd
pd.set_option('display.float_format', '{:,.4f}'.format)

import matplotlib.pyplot as plt

import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle

In [4]:
my_path = Path('/home/klarazetkin/Documents/yandex/module_2/project_3')
if my_path.is_dir():
    df = pd.read_csv('/home/klarazetkin/Documents/yandex/module_2/project_3/Churn.csv')
else:
    df = pd.read_csv('/datasets/Churn.csv')

In [5]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0000,0.0000,1,1,1,"101,348.8800",1
1,2,15647311,Hill,608,Spain,Female,41,1.0000,"83,807.8600",1,0,1,"112,542.5800",0
2,3,15619304,Onio,502,France,Female,42,8.0000,"159,660.8000",3,1,0,"113,931.5700",1
3,4,15701354,Boni,699,France,Female,39,1.0000,0.0000,2,0,0,"93,826.6300",0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0000,"125,510.8200",1,1,1,"79,084.1000",0


In [6]:
df.shape

(10000, 14)

In [7]:
df.isna().mean()

RowNumber         0.0000
CustomerId        0.0000
Surname           0.0000
CreditScore       0.0000
Geography         0.0000
Gender            0.0000
Age               0.0000
Tenure            0.0909
Balance           0.0000
NumOfProducts     0.0000
HasCrCard         0.0000
IsActiveMember    0.0000
EstimatedSalary   0.0000
Exited            0.0000
dtype: float64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


## Подготовка данных
### Переименование столбцов
Названия столбцов переведем из CamelCase в более привычный snake_case.

In [9]:
df = df.rename(columns={"RowNumber": "row_number", "CustomerId": "customer_id", "Surname": "surname", "CreditScore": "credit_score", "Geography": "geography", "Gender": "gender", "Age": "age", "Tenure": "tenure", "Balance": "balance", "NumOfProducts": "number_of_products", "HasCrCard": "has_credit_card", "IsActiveMember": "is_active_member", "EstimatedSalary": "estimated_salary", "Exited": "exited"})

In [10]:
df.columns

Index(['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'number_of_products',
       'has_credit_card', 'is_active_member', 'estimated_salary', 'exited'],
      dtype='object')

### Замена типов столбцов
Заменим тип столбцов `has_credit_card` и `is_active_member` на логический.

In [11]:
df['has_credit_card'] = df['has_credit_card'].astype('bool')
df['is_active_member'] = df['is_active_member'].astype('bool')

### Удаление лишних столбцов
Скорей всего, информация об имении и id клиента, а также `row_number` ничего не даст модели и будет мешать обучению. Уберем эти столбцы.

In [12]:
print(df.shape)
df = df.drop(['surname', 'customer_id', 'row_number'], axis=1)
print(df.shape)

(10000, 14)
(10000, 11)


### Заполнение NaN
Столбец `tenure` содержит пустые значения. Заполним их медианными.

In [13]:
print(df['tenure'].isna().sum())
df['tenure'] = df['tenure'].fillna(df['tenure'].median())
print(df['tenure'].isna().sum())

909
0


### Кодирование категориальных признаков техникой OHE
В колонках `gender` и `geography` содержатся категориальные признаки. Кодируем их техникой One-Hot Encoding, т.к. она подходит для всех моделей.

In [14]:
df.head()

,credit_score,geography,gender,age,tenure,balance,number_of_products,has_credit_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2.0000,0.0000,1,True,True,"101,348.8800",1
1,608,Spain,Female,41,1.0000,"83,807.8600",1,False,True,"112,542.5800",0
2,502,France,Female,42,8.0000,"159,660.8000",3,True,False,"113,931.5700",1
3,699,France,Female,39,1.0000,0.0000,2,False,False,"93,826.6300",0
4,850,Spain,Female,43,2.0000,"125,510.8200",1,True,True,"79,084.1000",0


In [15]:
print(df['geography'].unique())
print(df['gender'].unique())

['France' 'Spain' 'Germany']
['Female' 'Male']


In [16]:
df = pd.get_dummies(df, drop_first=True)

In [17]:
df.shape

(10000, 12)

In [18]:
df.columns

Index(['credit_score', 'age', 'tenure', 'balance', 'number_of_products',
       'has_credit_card', 'is_active_member', 'estimated_salary', 'exited',
       'geography_Germany', 'geography_Spain', 'gender_Male'],
      dtype='object')

### Масштабирование данных
Численные значения в колонках `balance` и `estimated_salary` на два порядка превышают значения в других численных колонках. Чтоб модель не считала эти данные более важными, масштабируем численные данные.

In [19]:
numeric = ['credit_score', 'age', 'tenure', 'balance', 'number_of_products', 'estimated_salary']

scaler = StandardScaler()
scaler.fit(df[numeric]) 

df[numeric] = scaler.transform(df[numeric])

In [20]:
df.head()

,credit_score,age,tenure,balance,number_of_products,has_credit_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
0,-0.3262,0.2935,-1.0862,-1.2258,-0.9116,True,True,0.0219,1,0,0,0
1,-0.4400,0.1982,-1.4486,0.1174,-0.9116,False,True,0.2165,0,0,1,0
2,-1.5368,0.2935,1.0878,1.3331,2.5271,True,False,0.2407,1,0,0,0
3,0.5015,0.0075,-1.4486,-1.2258,0.8077,False,False,-0.1089,0,0,0,0
4,2.0639,0.3889,-1.0862,0.7857,-0.9116,True,True,-0.3653,0,0,1,0


### Выделение обучающей, валидационной и тестовой выборок
Разделим имеющийся датасет на три части: обучающую (60%), валидационную (20%) и тестовую (20%) выборки. Укажем параметр `random_state=666` для обеспечения повторяемости результата.

In [21]:
df_train, df_valid_and_test = train_test_split(df, test_size=0.4, stratify=df['exited'], random_state=666)

In [22]:
df_valid, df_test = train_test_split(df_valid_and_test, test_size=0.5, stratify=df_valid_and_test['exited'], random_state=666)

In [23]:
# проверим размеры выборок
print(len(df_train))
print(len(df_valid))
print(len(df_test))

6000
2000
2000


In [24]:
# проверим, что % положительного класса целевого параметра представлены в выборках пропорционально
print(df_train['exited'].mean())
print(df_valid['exited'].mean())
print(df_test['exited'].mean())

0.20366666666666666
0.204
0.2035


### Определение целевого признака и других признаков
Целевой признак - факт ухода клиента, отраженный в колонке `df['exited']`.

In [25]:
features_train = df_train.drop('exited', axis=1)
target_train = df_train['exited']
features_valid = df_valid.drop('exited', axis=1)
target_valid = df_valid['exited']
features_test = df_test.drop('exited', axis=1)
target_test = df_test['exited']

## Создание и обучение моделей предсказания

### Исследование баланса классов
Целевой признак - факт ухода клиента. Исследуем баланс классов. Соотношение позитивного и негативного классов приблизительно 1 : 4, что довольно далеко от баланса 1 : 1. Попробуем обучить модели без учета дисбаланса, проверим их результаты и при необходимости переобучим с учетом дисбаланса.

In [26]:
df['exited'].value_counts()

0    7963
1    2037
Name: exited, dtype: int64

In [27]:
df['exited'].mean()

0.2037

### Создание и обучение моделей без учета дисбаланса классов
#### Логистическая регрессия
Логистическая регрессия, обученная без учета дисбаланса классов, показала результат `f1 = 0.3309608540925267`. Значение меры `AUC-ROC = 0.7454167282490886`. Значение целевого признака очень далеко от минимально приемлемого `f1 = 0.59`.

Создадим хэш `metrics` с финальными значениями метрик, куда будем записывать все значения метрик обученных моделей.

In [28]:
model = LogisticRegression(random_state=666, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_of_model = f1_score(target_valid, predicted_valid)
print('f1 =', f1_of_model)

f1 = 0.3309608540925267


In [29]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC =', auc_roc)

AUC-ROC = 0.7454167282490886


In [30]:
metrics = {'logistic_regression_unbalanced': {'f1': f1_of_model, 'auc_roc': auc_roc}}

#### Решающее дерево

У решающего дерева, обученного без учета дисбаланса классов, результат лучше: максимальное значение `f1 = 0.5796269727403156`и `auc_roc = 0.8185935806483398`. Лучшая модель имеет глубину 8 уровней.

In [31]:
best_model = None
best_f1 = 0
best_auc_roc = 0
tree_metrics = []

for depth in range(2, 21):
    model = DecisionTreeClassifier(max_depth=depth, random_state=666)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    
    tree_metrics.append([depth, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_f1 = f1_of_model
        best_model = model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
        

print()        
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

max_depth = 2 :   f1 = 0.48513302034428796  auc_roc = 0.7202841720859198
max_depth = 3 :   f1 = 0.5092024539877301  auc_roc = 0.7877869740861169
max_depth = 4 :   f1 = 0.4448398576512456  auc_roc = 0.8121435917331757
max_depth = 5 :   f1 = 0.5106382978723404  auc_roc = 0.8284714011232635
max_depth = 6 :   f1 = 0.5565476190476191  auc_roc = 0.8316051766183861
max_depth = 7 :   f1 = 0.5730337078651685  auc_roc = 0.826299081190265
max_depth = 8 :   f1 = 0.5796269727403156  auc_roc = 0.8185935806483398
max_depth = 9 :   f1 = 0.5633423180592991  auc_roc = 0.7921062112030742
max_depth = 10 :   f1 = 0.575197889182058  auc_roc = 0.7752064550694651
max_depth = 11 :   f1 = 0.526027397260274  auc_roc = 0.7493572334712779
max_depth = 12 :   f1 = 0.5178335535006604  auc_roc = 0.7387596684402404
max_depth = 13 :   f1 = 0.516795865633075  auc_roc = 0.7109051692284954
max_depth = 14 :   f1 = 0.503209242618742  auc_roc = 0.6932102300719283
max_depth = 15 :   f1 = 0.5054945054945055  auc_roc = 0.6935781

In [32]:
metrics['tree_unbalanced'] = {'f1': best_f1,  'auc_roc': 0.8185935806483398}

#### Случайный лес
Без учета дисбаланса классов лучший результат получился у модели случайного леса с 20 деревьями и глубиной 15: 
`f1 = 0.564885496183206` и `auc_roc = 0.8324888843728446`. 

In [33]:
best_depth = 1
best_model = None
best_f1 = 0
best_auc_roc = 0
forest_metrics = []

# Сначала обучим лес с 20 деревьями-оценщиками и найдем оптимальную глубину

for depth in range(2, 21):
    model = RandomForestClassifier(n_estimators=20, max_depth=depth, random_state=666)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    
    forest_metrics.append([depth, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_depth = depth
        best_model = model
        best_f1 = f1_of_model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
    
print()        
print('best_depth:', best_depth)
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

max_depth = 2 :   f1 = 0.10672853828306264  auc_roc = 0.7897037269681744
max_depth = 3 :   f1 = 0.3114754098360656  auc_roc = 0.8185142932801261
max_depth = 4 :   f1 = 0.3992537313432836  auc_roc = 0.8279387131737117
max_depth = 5 :   f1 = 0.4778156996587031  auc_roc = 0.849983680658193
max_depth = 6 :   f1 = 0.5339966832504146  auc_roc = 0.855137359592078
max_depth = 7 :   f1 = 0.5296052631578947  auc_roc = 0.8556638892994383
max_depth = 8 :   f1 = 0.5382165605095542  auc_roc = 0.8633901739087595
max_depth = 9 :   f1 = 0.5543307086614174  auc_roc = 0.8591517637205635
max_depth = 10 :   f1 = 0.5598755832037325  auc_roc = 0.8533838001280915
max_depth = 11 :   f1 = 0.5368916797488226  auc_roc = 0.8444451115873485
max_depth = 12 :   f1 = 0.5627836611195158  auc_roc = 0.838547824908858
max_depth = 13 :   f1 = 0.5579268292682926  auc_roc = 0.8452395248300325
max_depth = 14 :   f1 = 0.5449010654490106  auc_roc = 0.8458984567445069
max_depth = 15 :   f1 = 0.564885496183206  auc_roc = 0.832488

In [34]:
# Теперь подберем оптимальное количество деревьев-оценщиков
forest_metrics_2 = []
best_trees = 20

for trees in range (20, 250, 10):
    model = RandomForestClassifier(n_estimators = trees, 
                                  max_depth=best_depth, random_state=666)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("trees =", trees, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    forest_metrics_2.append([trees, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_trees = trees
        best_model = model
        best_f1 = f1_of_model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
        

        
print()    
print('best_depth:', best_depth)
print('best_trees:', best_trees)
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

trees = 20 :   f1 = 0.564885496183206  auc_roc = 0.8324888843728446
trees = 30 :   f1 = 0.5608628659476118  auc_roc = 0.8456413501330182
trees = 40 :   f1 = 0.5596330275229359  auc_roc = 0.849528740516307
trees = 50 :   f1 = 0.551617873651772  auc_roc = 0.8484156382402207
trees = 60 :   f1 = 0.5552147239263804  auc_roc = 0.8481793156961276
trees = 70 :   f1 = 0.5507692307692307  auc_roc = 0.8515678884619174
trees = 80 :   f1 = 0.5471406491499228  auc_roc = 0.852763357227313
trees = 90 :   f1 = 0.5490797546012269  auc_roc = 0.8524354308306238
trees = 100 :   f1 = 0.5446153846153846  auc_roc = 0.8529565720760666
trees = 110 :   f1 = 0.5493827160493827  auc_roc = 0.8527772132722435
trees = 120 :   f1 = 0.5555555555555556  auc_roc = 0.8534592385949353
trees = 130 :   f1 = 0.5564142194744977  auc_roc = 0.8533953468322002
trees = 140 :   f1 = 0.5564142194744977  auc_roc = 0.8537348199329983
trees = 150 :   f1 = 0.5541795665634675  auc_roc = 0.8525308835845896
trees = 160 :   f1 = 0.551937984

In [35]:
metrics['forest_unbalanced'] = {'f1': best_f1, 'auc_roc': 0.8324888843728446}

### Обучение моделей с учетом дисбаланса классов: весовая балансировка
Воспользуемся весовой балансировкой для снижения влияния дисбаланса классов. Укажем для моделей параметр `class_weight='balanced'`, обучим их и оценим результаты. 

#### Логистическая регрессия

Обучим модель логистической регрессии, указав в гиперпараметрах `class_weight='balanced'`. У новой модели `f1 = 0.4716981132075472` и `AUC-ROC = 0.7508960242388412`. 

Это выше, чем у модели, обученной без учета дисбаланса, но метрики все равно далеки от приемлемых.

In [36]:
model = LogisticRegression(random_state=666, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_of_model = f1_score(target_valid, predicted_valid)
print('f1 =', f1_of_model)

f1 = 0.4716981132075472


In [37]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC =', auc_roc)

AUC-ROC = 0.7508960242388412


In [38]:
metrics['logistic_regression_class_weight_balanced'] = {'f1': f1_of_model, 'auc_roc': auc_roc}

#### Решающее дерево
Обучим модель решающего дерева, указав в гиперпараметрах `class_weight='balanced'`. Наиболее удачная модель имеет глубину 6 уровней.

Эта модель показала лучшие результаты, по сравнению с моделью, обученной без учета дисбаланса классов: у новой модели `f1 = 0.5695970695970696` и `auc_roc = 0.8309500936052814`. Целевой показатель `f1` стал хуже, чем  у модели, обученной без учета дисбаланса (`f1 = 0.5796269727403156`).


In [39]:
best_model = None
best_f1 = 0
best_auc_roc = 0
tree_metrics = []

for depth in range(2, 21):
    model = DecisionTreeClassifier(max_depth=depth, random_state=666, class_weight='balanced')
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    
    tree_metrics.append([depth, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_f1 = f1_of_model
        best_model = model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
        

print()        
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

max_depth = 2 :   f1 = 0.4999999999999999  auc_roc = 0.7306869827076559
max_depth = 3 :   f1 = 0.4999999999999999  auc_roc = 0.7873766812001183
max_depth = 4 :   f1 = 0.5312783318223029  auc_roc = 0.8104531542516504
max_depth = 5 :   f1 = 0.5662337662337662  auc_roc = 0.835710414819194
max_depth = 6 :   f1 = 0.5690759377859103  auc_roc = 0.8303527441127205
max_depth = 7 :   f1 = 0.5698427382053655  auc_roc = 0.8357796950438465
max_depth = 8 :   f1 = 0.5523809523809523  auc_roc = 0.8077104271356783
max_depth = 9 :   f1 = 0.5465465465465464  auc_roc = 0.79218395901074
max_depth = 10 :   f1 = 0.5402061855670103  auc_roc = 0.7766320881367623
max_depth = 11 :   f1 = 0.5561497326203209  auc_roc = 0.7619192777613557
max_depth = 12 :   f1 = 0.5425188374596339  auc_roc = 0.7428880000492659
max_depth = 13 :   f1 = 0.5247747747747747  auc_roc = 0.726675657700266
max_depth = 14 :   f1 = 0.5287356321839081  auc_roc = 0.7158687124347226
max_depth = 15 :   f1 = 0.5053128689492327  auc_roc = 0.6973693

In [40]:
metrics['tree_class_weight_balanced'] = {'f1': best_f1, 'auc_roc': 0.8309500936052814}

In [41]:
metrics

{'logistic_regression_unbalanced': {'f1': 0.3309608540925267,
  'auc_roc': 0.7454167282490886},
 'tree_unbalanced': {'f1': 0.5796269727403156, 'auc_roc': 0.8185935806483398},
 'forest_unbalanced': {'f1': 0.564885496183206, 'auc_roc': 0.8324888843728446},
 'logistic_regression_class_weight_balanced': {'f1': 0.4716981132075472,
  'auc_roc': 0.7508960242388412},
 'tree_class_weight_balanced': {'f1': 0.5698427382053655,
  'auc_roc': 0.8309500936052814}}

#### Случайный лес
Обучим модель случайного леса, указав в гиперпараметрах `class_weight='balanced'`. 

Лучше всего себя показала модель случайного леса с 8 уровнями глубины и 80 деревьями-оценщиками. Ее метрики `f1 = 0.6148571428571429`  и `auc_roc = 0.855629249187112` - значения лучше, чем у лучшей модели леса, обученной без учета дисбаланса. 

In [42]:
best_depth = 1
best_model = None
best_f1 = 0
best_auc_roc = 0
forest_metrics = []

# Сначала обучим лес с 20 деревьями-оценщиками и найдем оптимальную глубину

for depth in range(2, 21):
    model = RandomForestClassifier(n_estimators=20, max_depth=depth, class_weight='balanced', random_state=666)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    
    forest_metrics.append([depth, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_depth = depth
        best_model = model
        best_f1 = f1_of_model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
    
print()        
print('best_depth:', best_depth)
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

max_depth = 2 :   f1 = 0.5352657004830919  auc_roc = 0.8041825241403094
max_depth = 3 :   f1 = 0.5548902195608783  auc_roc = 0.8191285779387132
max_depth = 4 :   f1 = 0.5787908820614469  auc_roc = 0.8360575857227313
max_depth = 5 :   f1 = 0.5892116182572614  auc_roc = 0.8445174709330969
max_depth = 6 :   f1 = 0.5892291446673706  auc_roc = 0.8437838703320524
max_depth = 7 :   f1 = 0.5884955752212389  auc_roc = 0.8497920053699872
max_depth = 8 :   f1 = 0.5805714285714285  auc_roc = 0.8462987424869445
max_depth = 9 :   f1 = 0.5983701979045402  auc_roc = 0.8518580956251849
max_depth = 10 :   f1 = 0.5778331257783312  auc_roc = 0.8526686742536211
max_depth = 11 :   f1 = 0.5823451910408431  auc_roc = 0.8495333591979507
max_depth = 12 :   f1 = 0.5572005383580081  auc_roc = 0.8485295657207605
max_depth = 13 :   f1 = 0.5787234042553191  auc_roc = 0.8409249063947187
max_depth = 14 :   f1 = 0.5627705627705628  auc_roc = 0.8481084959109271
max_depth = 15 :   f1 = 0.5565476190476191  auc_roc = 0.835

In [43]:
# Теперь подберем оптимальное количество деревьев-оценщиков
forest_metrics_2 = []
best_trees = 1

for trees in range (20, 250, 10):
    model = RandomForestClassifier(n_estimators = trees, 
                                  max_depth=best_depth, class_weight='balanced', random_state=666)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("trees =", trees, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    forest_metrics_2.append([trees, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_trees = trees
        best_model = model
        best_f1 = f1_of_model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
        

        
print()    
print('best_depth:', best_depth)
print('best_trees:', best_trees)
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

trees = 20 :   f1 = 0.5983701979045402  auc_roc = 0.8518580956251849
trees = 30 :   f1 = 0.5966981132075472  auc_roc = 0.8521252093802346
trees = 40 :   f1 = 0.6011834319526627  auc_roc = 0.8528010764607351
trees = 50 :   f1 = 0.6064209274673008  auc_roc = 0.8548148216573062
trees = 60 :   f1 = 0.6106508875739645  auc_roc = 0.8549426051827766
trees = 70 :   f1 = 0.6146458583433373  auc_roc = 0.8549518425460637
trees = 80 :   f1 = 0.6078665077473183  auc_roc = 0.855601537097251
trees = 90 :   f1 = 0.605263157894737  auc_roc = 0.8554937678589024
trees = 100 :   f1 = 0.6004784688995215  auc_roc = 0.8558724997536703
trees = 110 :   f1 = 0.6028708133971291  auc_roc = 0.8563158931914475
trees = 120 :   f1 = 0.6047619047619047  auc_roc = 0.856839343777712
trees = 130 :   f1 = 0.6054827175208581  auc_roc = 0.856688466844024
trees = 140 :   f1 = 0.6069295101553165  auc_roc = 0.8569055448812691
trees = 150 :   f1 = 0.6028708133971291  auc_roc = 0.8570071558774264
trees = 160 :   f1 = 0.605263157

In [44]:
metrics['forest_class_weight_balanced'] = {'f1': best_f1, 'auc_roc': 0.855629249187112}

In [45]:
metrics

{'logistic_regression_unbalanced': {'f1': 0.3309608540925267,
  'auc_roc': 0.7454167282490886},
 'tree_unbalanced': {'f1': 0.5796269727403156, 'auc_roc': 0.8185935806483398},
 'forest_unbalanced': {'f1': 0.564885496183206, 'auc_roc': 0.8324888843728446},
 'logistic_regression_class_weight_balanced': {'f1': 0.4716981132075472,
  'auc_roc': 0.7508960242388412},
 'tree_class_weight_balanced': {'f1': 0.5698427382053655,
  'auc_roc': 0.8309500936052814},
 'forest_class_weight_balanced': {'f1': 0.6146458583433373,
  'auc_roc': 0.855629249187112}}

### Обучение моделей с учетом дисбаланса классов: метод upsampling


#### Увеличение выборки методом upsampling
Воспользуемся альтернативным методом снижения влияния дисбаланса классов - увеличим выборку методом `upsampling`. Для этого определим функцию `upsample()` и передадим ей обучающую выборку. Баланс классов доведем примерно до 1 : 1.

In [46]:
def upsample(features, target, repeat):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled
    

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [47]:
print(target_upsampled.shape)
print(target_upsampled.mean())

(9666,)
0.5056900475894889


#### Логистическая регрессия

Обучим модель логистической регрессии на увеличенной выборке. У новой модели `f1 = 0.4731914893617022` и `AUC-ROC = 0.750965304463494`. Эта модель показала лучшие результаты, по сравнению с моделью, обученной без учета дисбаланса классов, и с моделью, обученной с учетом весовой балансировки. Целевой показатель по-прежнему далек от приемлемого `f1 = 0.59`.

In [48]:
model = LogisticRegression(random_state=666, solver='liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
f1_of_model = f1_score(target_valid, predicted_valid)
print('f1 =', f1_of_model)

f1 = 0.4731914893617022


In [49]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC-ROC =', auc_roc)

AUC-ROC = 0.750965304463494


In [50]:
metrics['logistic_regression_upsampled'] = {'f1': f1_of_model, 'auc_roc': auc_roc}
metrics

{'logistic_regression_unbalanced': {'f1': 0.3309608540925267,
  'auc_roc': 0.7454167282490886},
 'tree_unbalanced': {'f1': 0.5796269727403156, 'auc_roc': 0.8185935806483398},
 'forest_unbalanced': {'f1': 0.564885496183206, 'auc_roc': 0.8324888843728446},
 'logistic_regression_class_weight_balanced': {'f1': 0.4716981132075472,
  'auc_roc': 0.7508960242388412},
 'tree_class_weight_balanced': {'f1': 0.5698427382053655,
  'auc_roc': 0.8309500936052814},
 'forest_class_weight_balanced': {'f1': 0.6146458583433373,
  'auc_roc': 0.855629249187112},
 'logistic_regression_upsampled': {'f1': 0.4731914893617022,
  'auc_roc': 0.750965304463494}}

#### Решающее дерево

Обучим модель решающего дерева на увеличенной выборке. Наиболее удачная модель имеет глубину 6 уровней.

У новой модели `f1 = 0.5693296602387511` и  `auc_roc = 0.8294428638289487`. Целевой показатель `f1` немного хуже, чем у модели, обученной с весовой балансировкой (`f1 = 0.5695970695970696`), и значительно хуже, чем у модели, обученной без учета дисбаланса (`f1 = 0.5796269727403156`).


In [51]:
best_model = None
best_f1 = 0
best_auc_roc = 0
tree_metrics = []

for depth in range(2, 21):
    model = DecisionTreeClassifier(max_depth=depth, random_state=666)
    model.fit(features_upsampled, target_upsampled)
              
    predicted_valid = model.predict(features_valid)
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    
    tree_metrics.append([depth, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_f1 = f1_of_model
        best_model = model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
        

print()        
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

max_depth = 2 :   f1 = 0.4999999999999999  auc_roc = 0.7306869827076559
max_depth = 3 :   f1 = 0.4999999999999999  auc_roc = 0.7873766812001183
max_depth = 4 :   f1 = 0.5312783318223029  auc_roc = 0.8104531542516504
max_depth = 5 :   f1 = 0.5647263249348392  auc_roc = 0.8347343334318651
max_depth = 6 :   f1 = 0.5693296602387511  auc_roc = 0.8294428638289487
max_depth = 7 :   f1 = 0.5690454124189064  auc_roc = 0.8344356586855849
max_depth = 8 :   f1 = 0.5529075309818875  auc_roc = 0.8080914683712681
max_depth = 9 :   f1 = 0.5516549648946841  auc_roc = 0.7976632550004925
max_depth = 10 :   f1 = 0.5399792315680165  auc_roc = 0.7775296519361514
max_depth = 11 :   f1 = 0.5622317596566524  auc_roc = 0.7650715279830526
max_depth = 12 :   f1 = 0.5446623093681917  auc_roc = 0.7473542651985418
max_depth = 13 :   f1 = 0.5369738339021616  auc_roc = 0.7332957680559661
max_depth = 14 :   f1 = 0.5266821345707656  auc_roc = 0.7125070819785201
max_depth = 15 :   f1 = 0.5235849056603773  auc_roc = 0.713

In [52]:
metrics['tree_upsampled'] = {'f1': best_f1, 'auc_roc': 0.8294428638289487}
metrics

{'logistic_regression_unbalanced': {'f1': 0.3309608540925267,
  'auc_roc': 0.7454167282490886},
 'tree_unbalanced': {'f1': 0.5796269727403156, 'auc_roc': 0.8185935806483398},
 'forest_unbalanced': {'f1': 0.564885496183206, 'auc_roc': 0.8324888843728446},
 'logistic_regression_class_weight_balanced': {'f1': 0.4716981132075472,
  'auc_roc': 0.7508960242388412},
 'tree_class_weight_balanced': {'f1': 0.5698427382053655,
  'auc_roc': 0.8309500936052814},
 'forest_class_weight_balanced': {'f1': 0.6146458583433373,
  'auc_roc': 0.855629249187112},
 'logistic_regression_upsampled': {'f1': 0.4731914893617022,
  'auc_roc': 0.750965304463494},
 'tree_upsampled': {'f1': 0.5693296602387511, 'auc_roc': 0.8294428638289487}}

#### Случайный лес
Обучим модель случайного леса на увеличенной выборке. Наиболее точной из обученных этим методом моделей является модель случайного леса с 8 уровнями глубины и 20 деревьями-оценщиками.  

Ее метрики `f1 = 0.607621009268795` и `auc_roc = 0.8568809119125038`. Значение `f1` хуже, чем у лучшей модели леса с весовой балансировкой (`f1 = 0.6148571428571429`); значение `auc_roc` новой модели лучше, но этот показатель не целевой.

In [53]:
best_depth = 1
best_model = None
best_f1 = 0
best_auc_roc = 0
forest_metrics = []

# Сначала обучим лес с 20 деревьями-оценщиками и найдем оптимальную глубину

for depth in range(2, 21):
    model = RandomForestClassifier(n_estimators=20, max_depth=depth, random_state=666)
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid)
    
    f1_of_model = f1_score(target_valid, predicted_valid)
    
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)
    
    print("max_depth =", depth, ": ", end='  ')
    print('f1 =', f1_of_model, end='  ') 
    print('auc_roc =', auc_roc_of_model)
    
    forest_metrics.append([depth, f1_of_model, auc_roc_of_model])
    if f1_of_model > best_f1:
        best_depth = depth
        best_model = model
        best_f1 = f1_of_model
    if auc_roc_of_model > best_auc_roc:
        best_auc_roc = auc_roc_of_model
    
print()        
print('best_depth:', best_depth)
print('best_model:', best_model)
print('best_f1:', best_f1)
print('best_auc_roc:', best_auc_roc)

max_depth = 2 :   f1 = 0.5050136736554239  auc_roc = 0.7903595797615529
max_depth = 3 :   f1 = 0.5627376425855514  auc_roc = 0.8125731291260223
max_depth = 4 :   f1 = 0.5597014925373135  auc_roc = 0.8324788772292836
max_depth = 5 :   f1 = 0.5830039525691699  auc_roc = 0.8372153352547049
max_depth = 6 :   f1 = 0.5780000000000001  auc_roc = 0.8479145112818995
max_depth = 7 :   f1 = 0.5979166666666668  auc_roc = 0.8581487400236478
max_depth = 8 :   f1 = 0.607621009268795  auc_roc = 0.8568809119125038
max_depth = 9 :   f1 = 0.5966850828729281  auc_roc = 0.8502500246329688
max_depth = 10 :   f1 = 0.5926748057713651  auc_roc = 0.8473009964035866
max_depth = 11 :   f1 = 0.5819014891179839  auc_roc = 0.8504694120110355
max_depth = 12 :   f1 = 0.5971223021582734  auc_roc = 0.8423051224258549
max_depth = 13 :   f1 = 0.584652862362972  auc_roc = 0.8403206288796927
max_depth = 14 :   f1 = 0.5944584382871536  auc_roc = 0.8318207150950832
max_depth = 15 :   f1 = 0.5894465894465896  auc_roc = 0.84195

In [69]:
best_auc_roc = 0
best_f1 = 0
forest_models = []
cycle_index = 0


for this_criterion in ['gini', 'entropy']:
    for estimators in range(2, 60, 5):
        for depth in range(2, 30):
            for min_split in range(2, 5):
                for min_leaf in range(1, 5):
                    
                    print(cycle_index)
                    cycle_index += 1
                    
                    model = RandomForestClassifier(
                        random_state=12345, 
                        criterion=this_criterion,
                        n_estimators=estimators, 
                        max_depth=depth, 
                        min_samples_split=min_split, 
                        min_samples_leaf=min_leaf,
                        n_jobs=-1)
                    
                    model.fit(features_upsampled, target_upsampled)
                    predicted_valid = model.predict(features_valid)
                    
                    f1_of_model = f1_score(target_valid, predicted_valid)
                    
                    probabilities_valid = model.predict_proba(features_valid)
                    probabilities_one_valid = probabilities_valid[:, 1]
                    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)

                    forest_models.append([this_criterion, 
                                          estimators,
                                          depth,
                                          min_split,
                                          min_leaf,
                                          auc_roc_of_model, 
                                          f1_of_model])

                    if auc_roc_of_model > best_auc_roc:
                        best_auc_roc = auc_roc_of_model
                    if f1_of_model > best_f1:
                        best_f1 = f1_of_model
                    
print("Лучший auc_roc леса на валидационной выборке:", best_auc_roc)
print("Лучший f1_score леса на валидационной выборке:", best_f1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169


KeyboardInterrupt: 

In [55]:
metrics['forest_upsampled'] = {'f1': best_f1, 'auc_roc': 0.8568809119125038}
metrics

{'logistic_regression_unbalanced': {'f1': 0.3309608540925267,
  'auc_roc': 0.7454167282490886},
 'tree_unbalanced': {'f1': 0.5796269727403156, 'auc_roc': 0.8185935806483398},
 'forest_unbalanced': {'f1': 0.564885496183206, 'auc_roc': 0.8324888843728446},
 'logistic_regression_class_weight_balanced': {'f1': 0.4716981132075472,
  'auc_roc': 0.7508960242388412},
 'tree_class_weight_balanced': {'f1': 0.5698427382053655,
  'auc_roc': 0.8309500936052814},
 'forest_class_weight_balanced': {'f1': 0.6146458583433373,
  'auc_roc': 0.855629249187112},
 'logistic_regression_upsampled': {'f1': 0.4731914893617022,
  'auc_roc': 0.750965304463494},
 'tree_upsampled': {'f1': 0.5693296602387511, 'auc_roc': 0.8294428638289487},
 'forest_upsampled': {'f1': 0.607621009268795, 'auc_roc': 0.8568809119125038}}

#### Случайный лес
Попробуем подобрать гиперпараметры во вложенном цикле.


In [67]:
best_auc_roc_of_model = 0
best_forest_f1_score_valid = 0
forest_models = []
cycle_index = 0


for this_criterion in ['gini', 'entropy']:
    for estimators in range(2, 60, 5):
        for depth in range(2, 30):
            for min_split in range(2, 5):
                for min_leaf in range(1, 5):
                    
                    print(cycle_index)
                    cycle_index += 1
                    
                    model = RandomForestClassifier(
                        random_state=12345, 
                        criterion=this_criterion,
                        n_estimators=estimators, 
                        max_depth=depth, 
                        min_samples_split=min_split, 
                        min_samples_leaf=min_leaf,
                        n_jobs=-1)
                    
                    model.fit(features_upsampled, target_upsampled)
                    predicted_valid = model.predict(features_valid)
                    # forest_accuracy_score = model.score(features_valid, target_valid)
                    forest_f1_score = f1_score(target_valid, predicted_valid)
                    
                    probabilities_valid = model.predict_proba(features_valid)
                    probabilities_one_valid = probabilities_valid[:, 1]
                    auc_roc_of_model = roc_auc_score(target_valid, probabilities_one_valid)

                    forest_models.append([this_criterion, 
                                          estimators,
                                          depth,
                                          min_split,
                                          min_leaf,
                                          auc_roc_of_model, 
                                          forest_f1_score])

                    if auc_roc_of_model > best_auc_roc_of_model:
                        best_auc_roc_of_model = auc_roc_of_model
                    if forest_f1_score > best_forest_f1_score_valid:
                        best_forest_f1_score_valid = forest_f1_score
                    
print("Лучший auc_roc леса на валидационной выборке:", best_auc_roc_of_model)
print("Лучший f1_score леса на валидационной выборке:", best_forest_f1_score_valid)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


KeyboardInterrupt: 

Показатели лучших из подобранных моделей: 
    

In [66]:
forest_models = pd.DataFrame(forest_models, 
                           columns=['criterion', 'estimators', 'depth', 
                                    'min_sample_split', 'min_sample_leaf', 
                                    'auc_roc', 'f1_score'])


display(forest_models[forest_models['auc_roc'] == forest_models['auc_roc'].max()])
display(forest_models[forest_models['f1_score'] == forest_models['f1_score'].max()])

,criterion,estimators,depth,min_sample_split,min_sample_leaf,auc_roc,f1_score
108,gini,2,11,2,1,0.7945,0.5604


,criterion,estimators,depth,min_sample_split,min_sample_leaf,auc_roc,f1_score
73,gini,2,8,2,2,0.7835,0.5734
77,gini,2,8,3,2,0.7835,0.5734
81,gini,2,8,4,2,0.7835,0.5734


### Выбор и финальное тестирование лучшей модели
#### Выбор лучшей модели
Рассмотрим характеристики лучших из обученных каждым методом моделей и выберем самую лучшую. Лучший целевой показатель у модели случайного леса, обученного с учетом балансировки классов. Вот эта модель:
> RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=80, n_jobs=None, oob_score=False,
                       random_state=666, verbose=0, warm_start=False)

In [56]:
metrics

{'logistic_regression_unbalanced': {'f1': 0.3309608540925267,
  'auc_roc': 0.7454167282490886},
 'tree_unbalanced': {'f1': 0.5796269727403156, 'auc_roc': 0.8185935806483398},
 'forest_unbalanced': {'f1': 0.564885496183206, 'auc_roc': 0.8324888843728446},
 'logistic_regression_class_weight_balanced': {'f1': 0.4716981132075472,
  'auc_roc': 0.7508960242388412},
 'tree_class_weight_balanced': {'f1': 0.5698427382053655,
  'auc_roc': 0.8309500936052814},
 'forest_class_weight_balanced': {'f1': 0.6146458583433373,
  'auc_roc': 0.855629249187112},
 'logistic_regression_upsampled': {'f1': 0.4731914893617022,
  'auc_roc': 0.750965304463494},
 'tree_upsampled': {'f1': 0.5693296602387511, 'auc_roc': 0.8294428638289487},
 'forest_upsampled': {'f1': 0.607621009268795, 'auc_roc': 0.8568809119125038}}

In [57]:
best_f1 = 0
best_model_name = None

for item in metrics.items():
    if item[1]['f1'] > best_f1:
        best_f1 = item[1]['f1']
        best_model_name = item[0]
        
print(best_model_name, metrics[best_model_name])

forest_class_weight_balanced {'f1': 0.6146458583433373, 'auc_roc': 0.855629249187112}


#### Финальное тестирование
Проведем финальное тестирование лучшей модели - модели случайного леса:

> best_model: RandomForestClassifier(class_weight='balanced', max_depth=8, n_estimators=80, random_state=666)

Модель показала удовлетворительные результаты на тестовой и валидационной выборках:

f1 на обучающей выборке:  0.7182569496619083

f1 на валидационной выборке:  0.6148571428571429

f1 на тестовой выборке:  0.6131549609810478

AUC-ROC на обучающей выборке:  0.9313193174663743

AUC-ROC на валидационной выборке:  0.855629249187112

AUC-ROC на тестовой выборке:  0.8506395455547998

In [58]:
best_model = RandomForestClassifier(n_estimators = 80, 
                                  max_depth=8, class_weight='balanced', random_state=666)
print(best_model)
best_model.fit(features_train, target_train)
predicted_train = best_model.predict(features_train) 
predicted_valid = best_model.predict(features_valid)
predicted_test = best_model.predict(features_test)
f1_train = f1_score(target_train, predicted_train)
f1_valid = f1_score(target_valid, predicted_valid)
f1_test = f1_score(target_test, predicted_test)


probabilities_train = best_model.predict_proba(features_train)
probabilities_one_train = probabilities_train[:, 1]
auc_roc_train = roc_auc_score(target_train, probabilities_one_train)

probabilities_valid = best_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc_valid = roc_auc_score(target_valid, probabilities_one_valid)

probabilities_test = best_model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc_test = roc_auc_score(target_test, probabilities_one_test)

print("Наилучшая модель")
print("f1 на обучающей выборке: ", f1_train)
print("f1 на валидационной выборке: ", f1_valid)
print("f1 на тестовой выборке: ", f1_test)

print("AUC-ROC на обучающей выборке: ", auc_roc_train)
print("AUC-ROC на валидационной выборке: ", auc_roc_valid)
print("AUC-ROC на тестовой выборке: ", auc_roc_test)


RandomForestClassifier(class_weight='balanced', max_depth=8, n_estimators=80,
                       random_state=666)
Наилучшая модель
f1 на обучающей выборке:  0.7239031770045385
f1 на валидационной выборке:  0.5983701979045402
f1 на тестовой выборке:  0.6093750000000001
AUC-ROC на обучающей выборке:  0.9315692011736827
AUC-ROC на валидационной выборке:  0.8548102029756626
AUC-ROC на тестовой выборке:  0.8521772928552588


In [59]:
output_table = pd.DataFrame(
    [['Обучающая выборка', f1_train, auc_roc_train],
    ['Валидационная выборка', f1_valid, auc_roc_valid],
    ['Тестовая выборка', f1_test, auc_roc_test]],
    columns=['Выборка', 'F1-мера', 'AUC-ROC мера']
)
display(output_table)

,Выборка,F1-мера,AUC-ROC мера
0,Обучающая выборка,0.7239,0.9316
1,Валидационная выборка,0.5984,0.8548
2,Тестовая выборка,0.6094,0.8522


## Вывод
Для решения задачи предсказания ухода клиента банка были построены и обучены модели трех типов: решающее дерево, случайный лес и логистическая регрессия. Был произведен подбор гиперпараметров моделей и выбрана лучшая модель для каждого типа. 

Модели были обучены сначала без учета дисбаланса классов в датасете, а после с учетом этого дисбаланса. Для того, чтоб исправить дисбаланс, были испробованы два метода: 

* обучающая выборка была увеличена методом `upsampling`, доля положительного (редкого) класса до ведена до 50%;
* модели были обучены с весовой балансировкой (при инициализации модели указан гиперпараметр `class_weight='balanced'`).

Модели, обученные с учетом дисбаланса, показали лучшие результаты в сравнении с моделями, обученными без учета дисбаланса классов, хотя апсамплинг улучшил целевой показатель только для логистической регрессии. 

Лучше всего себя показала модель решающего леса с гиперпараметрами: `max_depth=8`, `n_estimators=80`, `class_weight='balanced'`, обученная на валидационной выборке, не измененной методом `upsampling`.

Метрики этой модели приведены в таблице:

In [60]:
display(output_table)

,Выборка,F1-мера,AUC-ROC мера
0,Обучающая выборка,0.7239,0.9316
1,Валидационная выборка,0.5984,0.8548
2,Тестовая выборка,0.6094,0.8522


Модель рекомендуется дообучить и начать использовать в бизнесе.